In [5]:
# module
import pysftp
import psycopg2
import pandas as pd
import numpy as np
import zipfile
import datetime

In [7]:
# 변수 선언
start = '2021-12'
end = '2022-06'
table = 'm1.kt_bldg_trading_area'
filename = 'dp_frc_basexy_ftc_m_sum_'
dir = 'C:\oasis\KT_data'

In [ ]:
# DB Connect
conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)

In [ ]:
# Bulk Insert
def execute_mogrify(conn, df, table):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", tup).decode('utf8') for tup in tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [ ]:
# SFTP Connect
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

with pysftp.Connection(
        '14.63.143.72',
        port = 2222,
        username = 'oasisuser',
        password = '4FTPuser1!',
        cnopts = cnopts) as sftp:

    for i in pd.date_range(start, end, freq = 'MS'):
        # file 내려 받기
        sftp.get(filename + i.strftime("%Y%m") + '.zip', dir + filename + i.strftime("%Y%m") + '.zip')
        # 압축 풀기
        zipfile.ZipFile(dir + filename + i.strftime("%Y%m") + '.zip').extractall(dir)
        # read
        df = pd.read_csv(dir + filename + i.strftime("%Y%m") + '.csv', sep='|', low_memory=False)
        df.replace({np.nan: None})
        # insert
        for j in range(0, len(df), 100000):
            tmp = df[j:j + 100000]
            print(datetime.datetime.now(), ' : ', j)
            execute_mogrify(conn, tmp, table)
            tmp = pd.DataFrame()